In [ ]:
import numpy as np
import math
import random

In [ ]:
#defining the cdf function of wangs paper Fx(a)
def initial_cdf(x1,n):

    cdf_initial = []
    x = np.arange(-5,8,0.1) # will give values between -5 and 8 with a seperation of 0.1
    for a in x:
        suma = np.sum([i < a for i in x1])/n
        print(suma)
        cdf_initial.append(suma)
    return cdf_initial

In [2]:
def update_CDF(cdf_old,x1n,n):
    cdf_new = []
    x = np.arange(-5,8,0.1)
    count = 0
    for a in x:
        suma = (((n-1)/n) * cdf_old[count]) + ((x1n < a)/n)
        cdf_new.append(suma)
        count = count + 1
        #print(type(suma))
    #print(count)
    return cdf_new

In [1]:
def computeKS_seq(cdfx1,cdfx2,n):
    x = np.arange(-5,8,0.1)
    count = 0
    dsum_updated = []
    for j in x:
        dsum_updated.append(abs(cdfx1[count] - cdfx2[count]))
        count = count + 1
    #print(dsum_updated)
    return max(dsum_updated)

In [ ]:
###########     Algorithm 1 KMED-SEQ: Clustering when K is known    #########

M = 12  # total no of streams
K = 2   # no of clusters
#Mk = 5  # no of streams per cluster
correct = [[0,1,2,3,4,5,6,7,8],[9,10,11]]
means = [0.4,0.55,0.7,0.85,1,1.15,1.3,1.45,1.6,1.85,2,2.15]

print(correct,means)

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11]] [0.4, 0.55, 0.7, 0.85, 1, 1.15, 1.3, 1.45, 1.6, 1.85, 2, 2.15]


In [ ]:
import math
constvec = [1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5]
Pe = np.zeros(len(constvec))
lnPe=np.zeros(len(constvec))
expsamples=np.zeros(len(constvec))
print(Pe,lnPe,expsamples)

for ii in range(len(constvec)):
    const = constvec[ii]
    iter_sum = 0
    error = 0
    total = 0
    while error < 30:
        ##############  INITIALISATION  ################
        # cluster indices 0 to M-1 and an empty list lst
        clusters = [[i] for i in range(M)]
        lst = [[] for i in range(M)]
        cdf_lst = [[] for i in range(M)]
        #print(1,cdf_lst)

        # generate first 2 samples to initialize MMD estimate (KSD estimate)
        nn = 2
        X = np.asarray([np.random.normal(means[j],1,nn) for j in range(M)])

        ####    Calculating initial CDFs    ####

        for i in range(M):
            cdf_lst[i] = initial_cdf(X[i,:],nn)
        #print('Initial cdf',cdf_lst)

        ####    Calculating initial KSDs    ####
        lst_dseq = np.zeros((M,M))
        for i in range(1,M):
            for j in range(i):
                lst_dseq[i][j] = computeKS_seq(cdf_lst[i],cdf_lst[j],nn)
                lst_dseq[j][i] = lst_dseq[i][j]

        ####    Calculating initial MMDs between each pair of sequences    ####

        # lst_dseq = np.zeros((M,M))
        # for i in range(1,M):
        #     for j in range(i):
        #         lst_dseq[i][j] = computeMMD(X[i,:],X[j,:],nn)
        #         lst_dseq[j][i] = lst_dseq[i][j]
        #print(lst_fixed)



        ##### TEST-STATISTIC Tn INITIALISATION #####
        Tn = 0
        #print(np.min(lst_index,axis = 0))               #it will a column matrix of all 0
        #print(np.min(lst_index,axis = 1))                #it will a row matrix of all 0

        #minval = np.min(lst_fixed[np.nonzero(lst_fixed)])         #will give you the non zero min value of the matrix
        #print(minval)

        #[min_i,min_j] = np.where(lst_fixed==minval)       #will give the index of the  min value of the matrix
        #print([min_i,min_j])

        #Tn = minval    #minimum inter-cluster distance
        #print('Minimum inter-cluster distance after iteration ', nn, ':', Tn)

        while Tn < (const/(nn**0.5)):
            nn = nn + 1

            lst_dc = [[] for i in range(M)]
            lst_dc = [[lst_dseq[i][j] for j in range(i+1)] for i in range(M)]
            for i in range(M):
                for j in range(M):
                    if i<j:
                        lst_dc[i].append(lst_dc[j][i])
            #print(lst_new)
            #####
            # Code for SLINK
            clusters = [[i] for i in range(M)]
            # Merge closest clusters till number of clusters is equal to K
            while len(clusters)>K:

                ####    FIND MINIMUM    ####

                #mi = sys.float_info.max
                mi = 100
                for i in range(len(lst_dc)):
                    for j in range(i+1):
                        if i != j:
                            if lst_dc[i][j] < mi:
                                mi = lst_dc[i][j]
                                a,b = [i,j]
                #print("the values of a and b are :", a,b)
                clusters[b].extend(clusters[a])  #add the elements of a to b at the end
                del clusters[a]                   #delete cluster a
                #print(clusters)

                ####    MERGE ROWS  ####
                #print("the lst[b] is ",lst[b])
                #print("the lst[a] is ",lst[a])
                for i in range(len(lst_dc[b])):        #merge rows 'a' and 'b'
                    if min(lst_dc[a][i],lst_dc[b][i]) != 0:
                        lst_dc[b][i] = min(lst_dc[a][i],lst_dc[b][i])
                del lst_dc[a]

                ####    UPDATE ROWS     ####
                for i in range(len(lst_dc)):       #update all other rows
                    if i!=b:
                        lst_dc[i][b] = min(lst_dc[i][a],lst_dc[i][b])
                    del lst_dc[i][a]   #new statement

                #print('dist matrix', lst)
                #print('dist matrix 2', dist)
                #print('clusters',clusters)

            #print('Clusters after iteration ', nn, ':', clusters)

            mi = 100
            for i in range(len(lst_dc)):
                for j in range(i+1):
                    if i != j:
                        if lst_dc[i][j] < mi:
                            mi = lst_dc[i][j]

            #print(lst_dc)
            Tn = mi
            #print(nn, Tn)

            # generate one more sample for each data stream
            z = np.asarray([np.random.normal(means[j],1,1) for j in range(M)])  #creates a 1 dim array with means and SD 1
            X = np.append(X,z,axis=1)       # new samples   #axis =1 means it will add the values of z along column wise

            # update CDFs

            for i in range(M):
                cdf_lst[i] = update_CDF(cdf_lst[i],z[i][0],nn)
            #print('Updated cdfs',type(cdf_lst))

            #Update KSDs between each pair of data streams
            lst_dseq_1=np.zeros((M,M))
            for i in range(M):
                for j in range(i):
                    lst_dseq_1[i,j] = computeKS_seq(cdf_lst[i],cdf_lst[j],nn)
                    #lst_dseq_1[i,j] = math.sqrt(((lst_dseq[i][j]*(nn-1)/nn)**2) + update(X[i,:],X[j,:],nn))      # updated distances
                    lst_dseq_1[j,i]=lst_dseq_1[i,j]
            for i in range(M):
                for j in range(M):
                    lst_dseq[i,j] =lst_dseq_1[i,j]



        # print('Minimum inter-cluster distance after iteration ', nn, ':', Tn)
        # print('Clusters after iteration ', nn, ':', clusters)
        # print('Errors', error)
        #print('Total', total)

        # Check if clustering is correct and update errors and number of samples taken
        iter_sum += nn
        # CHECKING IF IT'S CORRECT
        clusters.sort()
        C_sort = []
        for x in clusters:
            x.sort()
            C_sort.append(x)

        if C_sort != correct:
            error = error + 1
        total = total + 1

        #print(C_sort)



    Pe[ii] = error/total
    lnPe[ii] = math.log(Pe[ii])
    expsamples[ii]= iter_sum/total
    print(lnPe[ii],expsamples[ii])

print(total,expsamples,lnPe)
print(iter_sum / total)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-0.03278982282299084 4.225806451612903
-0.06453852113757118 19.65625
-0.44895022004790314 86.61702127659575
-0.262364264467491 202.17948717948718
-0.7884573603642702 483.43939393939394
-1.1735135968412274 788.340206185567
-1.1096621248546947 1172.0
-1.8510760463844744 1609.3664921465968
-2.5041644663924147 2112.9536784741144
-3.0696021221204464 2668.504643962848
-3.4404180948154366 3291.0202991452993
-4.091562919260224 3883.4105849582174
1795 [   4.22580645   19.65625      86.61702128  202.17948718  483.43939394
  788.34020619 1172.         1609.36649215 2112.95367847 2668.50464396
 3291.02029915 3883.41058496] [-0.03278982 -0.06453852 -0.44895022 -0.26236426 -0.78845736 -1.1735136
 -1.10966212 -1.85107605 -2.50416447 -3.06960212 -3.44041809 -4.09156292]
3883.4105849582174
